In [1]:
import deepspeech
from datasets import load_dataset
import numpy as np
import pandas as pd
import soundfile as sf
import time


In [2]:
PATH_TRANSCRIPTIONS = "transcriptions.csv"
pre_train_file = 'deepspeech-0.9.3-models.pbmm'
beam_width = 300
model = deepspeech.Model(pre_train_file)

In [3]:
# define function to read in sound file
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch
    
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
ds = ds.map(map_to_array)

Reusing dataset librispeech_asr (/home/lucasagrizzi/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)
Loading cached processed dataset at /home/lucasagrizzi/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc/cache-361ab9f8e626e5d4.arrow


In [ ]:

# initialize constants
transcriptions = []
error = []
inference_time = []
texts = []

len_ds = len(ds["text"])
BATCH_SIZE = 1


for i in range(len_ds//BATCH_SIZE):

    # import audio
    texts.append(ds['text'][i])
    audio = ds["speech"][i]
    audio = np.array(audio)
    a = audio + np.abs(audio.min())
    a = (a * 1024).astype(np.int16)
    # count time of inference
    start = time.time()
    
    transcription = model.stt(a)

    # store time of inference
    inference_time.append(time.time() - start)

    # store transcription
    transcriptions.append(transcription)

all_transcriptions = list(np.array(transcriptions).reshape(-1))

In [ ]:
# df_transcriptions = pd.read_csv(PATH_TRANSCRIPTIONS, index_col=0)
df_transcriptions = pd.DataFrame()

df_transcriptions[pre_train_file] = all_transcriptions
df_transcriptions[pre_train_file] = df_transcriptions[pre_train_file].apply(lambda x: x.upper())

df_transcriptions[pre_train_file + "_inf_time"] = inference_time
df_transcriptions['ground_truth'] = texts
df_transcriptions

In [ ]:
df_transcriptions.to_csv(PATH_TRANSCRIPTIONS)

In [ ]:
# import numpy as np
# import wave
# filename = 'example.wav'
# w = wave.open(filename, 'r')
# rate = w.getframerate()
# frames = w.getnframes()
# buffer = w.readframes(frames)
# print(rate)
# print(model.sampleRate())
# type(buffer)
# data16 = np.frombuffer(buffer, dtype=np.int16)
# type(data16)
# print(data16.shape)
# text = model.stt(data16)
# print(text)